In [30]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

In [31]:
df = pd.read_csv("../datasets/covid_toy.csv")

df.sample(5)

,age,gender,fever,cough,city,has_covid
19,42,Female,NaN,Strong,Bangalore,Yes
49,44,Male,104.0,Mild,Mumbai,No
44,20,Male,102.0,Strong,Delhi,No
17,40,Female,98.0,Strong,Delhi,No
66,51,Male,104.0,Mild,Kolkata,No


In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['has_covid']),
    df['has_covid'],
    test_size=0.2
)

X_train

,age,gender,fever,cough,city
10,75,Female,NaN,Mild,Delhi
16,69,Female,103.0,Mild,Kolkata
73,34,Male,98.0,Strong,Kolkata
18,64,Female,98.0,Mild,Bangalore
1,27,Male,100.0,Mild,Delhi
...,...,...,...,...,...
64,42,Male,104.0,Mild,Mumbai
14,51,Male,104.0,Mild,Bangalore
5,84,Female,NaN,Mild,Bangalore
24,13,Female,100.0,Strong,Kolkata


In [33]:
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[['fever']])

X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [34]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [35]:
ohe = OneHotEncoder(drop='first')

X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])

X_train_gender_city.shape

(80, 4)

In [36]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [37]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

X_train_transformed.shape

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 2 has 0 dimension(s)

## Best way to perform column transformation

In [ ]:
transformer = ColumnTransformer(
    transformers=[
        ('tnf1', SimpleImputer(), ['fever']),
        ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
        ('tnf3', OneHotEncoder(drop='first'), ['gender', 'city'])
    ],
    remainder='passthrough'
)

transformer.fit_transform(X_train).shape

(80, 7)

In [ ]:
transformer.transform(X_test).shape

(20, 7)